**Machine Learning Model Goal**: Predict the gains in terms of system cost and/or CO2 emissions of having VPP assets in the power grid of North India

> Indented block



In [ ]:
from google.colab import drive
WORK_DRIVE = '/content/drive'
drive.mount(WORK_DRIVE)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from pickle import dump

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
import os

WORK_AREA = WORK_DRIVE + '/MyDrive/Ansh Aggarwal/Models/Unit Commitment and Economic Dispatch'
#Change the working directory to the folder
os.chdir(WORK_AREA)
inputs_path = './inputs/'
beopt_path = './BEopt/'

In [ ]:
power_plant_data = pd.read_csv(inputs_path + 'power_plant_data.csv')
spec_beopt_data = pd.read_csv(beopt_path + 'TemperatureCSV.csv')

X = pd.read_csv('X_Matrix.csv')
Y = pd.read_csv('Y_Matrix.csv')

parse_dates = ['Hourly_Demand_Date']
hourly_demand = pd.read_csv(beopt_path + 'Normalize_BEopt_and_Demand.csv', parse_dates=parse_dates)


In [ ]:
X.columns

Index(['hour_76_demand', '01-01', '01-02', '01-03', '01-04', '01-05', '01-06',
       '01-07', '01-08', '01-09',
       ...
       'Unnamed: 356', 'Unnamed: 357', 'Unnamed: 358', 'Unnamed: 359',
       'Unnamed: 360', 'Unnamed: 361', 'Unnamed: 362', 'Unnamed: 363',
       'Unnamed: 364', 'Unnamed: 365'],
      dtype='object', length=366)

Code to Get Temperature Values and Added into Matrix

In [ ]:
months = {
    1: range(0, 744),    # Jan
    5: range(2880, 3624),  # May
    6: range(3625, 4345),  # June
    12: range(8016, 8760)  # Dec
}

new_df = []

for hour in range(24):
    hour_values = []
    for month, indices in months.items():
        hour_values.extend(spec_beopt_data.loc[indices[hour::24], "Avg_73_PV"].values)
    new_df.append(hour_values)

reshaped_df = pd.DataFrame(new_df)

reshaped_df.to_csv(beopt_path + 'Avg_76_PV.csv', index=False)




In [ ]:
df_avg_70_DB = pd.read_csv(beopt_path + 'Avg_70_DB.csv')
df_avg_73_DB = pd.read_csv(beopt_path + 'Avg_73_DB.csv')
df_avg_76_DB = pd.read_csv(beopt_path + 'Avg_76_DB.csv')
df_avg_70_OD = pd.read_csv(beopt_path + 'Avg_70_OD.csv')
df_avg_73_OD = pd.read_csv(beopt_path + 'Avg_73_OD.csv')
df_avg_76_OD = pd.read_csv(beopt_path + 'Avg_76_OD.csv')
df_avg_70_HR = pd.read_csv(beopt_path + 'Avg_70_HR.csv')
df_avg_73_HR = pd.read_csv(beopt_path + 'Avg_73_HR.csv')
df_avg_76_HR = pd.read_csv(beopt_path + 'Avg_76_HR.csv')
df_avg_70_PV = pd.read_csv(beopt_path + 'Avg_70_PV.csv')
df_avg_73_PV = pd.read_csv(beopt_path + 'Avg_73_PV.csv')
df_avg_76_PV = pd.read_csv(beopt_path + 'Avg_76_PV.csv')

Code to get the Demand at each Cooling Point formatted

In [ ]:
def get_month(row):
  month = row.Hourly_Demand_Date.month
  return month


def get_month_day(row):
  month = row.Hourly_Demand_Date.month
  day = row.Hourly_Demand_Date.day
  month_day = '{:02}-{:02}'.format(month, day)
  return month_day

def get_hour(row):
  hour = row.Hourly_Demand_Date.hour
  day = row.Hourly_Demand_Date.day
  return hour

hourly_demand['month'] = hourly_demand.apply(lambda x: get_month(x), axis=1)

month_mask = (hourly_demand.month == 1) | (hourly_demand.month == 5) | (hourly_demand.month == 6) | (hourly_demand.month == 12)
hourly_demand_new = hourly_demand[month_mask].copy()

hourly_demand_new = hourly_demand_new[['Hourly_Demand_Date',	'Demand_70_F_MW']]

hourly_demand_new['day_month'] = hourly_demand.apply(lambda x: get_month_day(x), axis=1)
hourly_demand_new['hour'] = hourly_demand.apply(lambda x: get_hour(x), axis=1)


In [ ]:
table70F = pd.pivot_table(hourly_demand_new, values='Demand_70_F_MW', index=['hour'],
                       columns=['day_month'], aggfunc=np.mean)

table70F.to_csv(beopt_path + 'placeholder.csv', index=False)
table70F


day_month,01-01,01-02,01-03,01-04,01-05,01-06,01-07,01-08,01-09,01-10,...,12-22,12-23,12-24,12-25,12-26,12-27,12-28,12-29,12-30,12-31
hour,,,,,,,,,,,,,,,,,,,,,
0.0,8030.163366,7657.371685,10329.578600,10119.714350,9918.783713,17262.158690,9463.866831,9943.932247,10278.164030,9514.991012,...,6561.740660,6491.777716,6519.804845,6174.848695,5746.934474,6423.516851,6572.264298,11308.499670,14114.750140,14447.038380
1.0,6930.870173,5831.939034,6687.073065,6145.284559,5674.331280,7786.483723,6157.057542,5323.029564,5458.043302,6091.136295,...,5539.822040,5640.269309,5562.145969,5324.326535,4422.150666,5341.429534,4947.398155,9658.968056,11197.481030,12113.030090
2.0,6175.497823,6091.843496,8360.815155,8362.659451,7515.974204,7723.258076,7408.497169,7408.113068,8227.996585,7811.004830,...,4903.632236,5007.081590,4926.775257,4706.736433,4311.986909,5233.716209,4917.220021,8122.709195,10727.333950,10738.155010
3.0,6246.787762,5511.690837,5963.863670,5529.643900,5311.760444,4727.768307,5814.501637,5039.479028,5310.393757,5440.926354,...,4674.311599,5037.184750,4784.086900,4612.603680,3852.221281,4458.967469,4296.951751,8449.523854,9432.701789,10737.047820
4.0,6217.178065,6071.540379,8362.906212,8005.473456,7308.470333,7022.004568,7728.215361,7824.198918,7499.802671,7656.247158,...,4745.368597,4913.320685,4969.790740,4680.114850,4444.370458,4919.758680,5015.651833,8124.033456,10266.353660,9789.490393
5.0,6956.370680,5628.047939,5921.340332,5717.655772,5504.057829,5473.998002,5708.249290,5053.455486,6216.537368,5645.127357,...,4925.060425,5089.883782,5093.353477,4820.050459,3942.329879,4708.983019,4532.874649,8473.056939,10258.165460,10963.609640
6.0,8837.970455,8039.391723,9932.164504,9315.828675,8772.728237,8868.568100,9800.487081,9570.883931,9697.734475,9056.453078,...,6294.796995,6362.330675,6551.410547,6229.783964,5643.748133,6859.808747,6584.417957,10621.844580,12078.240840,11756.906030
7.0,13591.710590,10755.510040,11005.792960,10592.832530,9972.875947,9663.738389,10263.162310,9618.712806,10282.273580,10317.935650,...,9199.903286,9388.036209,9408.319928,9055.490755,7717.162343,8661.744395,8743.195363,15929.469130,19069.816740,17839.642250
8.0,13459.529090,13050.908770,15762.541180,13799.223840,14457.345480,13385.198880,16238.545340,14062.329000,13936.739130,13895.362160,...,8786.487673,9025.478413,9156.984430,9168.639277,9473.301706,10183.346640,9844.912100,16682.553670,19600.459000,20706.289420


In [ ]:
#table70F.T
from sklearn import preprocessing

hours = 24
demand_column_names = ["demand_hr_{}".format(x) for x in range(hours)]

scaler_demand = preprocessing.MinMaxScaler()
d = scaler_demand.fit_transform(table70F.T)
scaled_demand = pd.DataFrame(d, columns=demand_column_names)
scaled_demand.head()

,demand_hr_0,demand_hr_1,demand_hr_2,demand_hr_3,demand_hr_4,demand_hr_5,demand_hr_6,demand_hr_7,demand_hr_8,demand_hr_9,...,demand_hr_14,demand_hr_15,demand_hr_16,demand_hr_17,demand_hr_18,demand_hr_19,demand_hr_20,demand_hr_21,demand_hr_22,demand_hr_23
0,0.250408,0.383149,0.288236,0.365444,0.369953,0.409463,0.566915,0.599055,0.330089,0.361597,...,0.455266,0.387230,0.430118,0.476205,0.622722,0.407240,0.636608,0.445882,0.535318,0.352708
1,0.220139,0.268730,0.278986,0.287109,0.350493,0.274975,0.460179,0.391473,0.312463,0.269490,...,0.413692,0.214781,0.432347,0.451269,0.592872,0.273666,0.547293,0.352123,0.579058,0.269112
2,0.437109,0.357765,0.529862,0.335294,0.656674,0.304670,0.713162,0.409791,0.429430,0.429872,...,0.476277,0.218190,0.435382,0.487810,0.628238,0.302724,0.644870,0.349480,0.531370,0.303045
3,0.420069,0.301355,0.530066,0.289022,0.608912,0.284047,0.630784,0.379567,0.344742,0.254582,...,0.438795,0.201982,0.426816,0.368833,0.553955,0.272003,0.546220,0.279449,0.471113,0.827582
4,0.403754,0.252320,0.436449,0.265804,0.515776,0.262421,0.558195,0.334192,0.373130,0.252489,...,0.433613,0.205007,0.410814,0.503707,0.559808,0.267247,0.540367,0.275704,0.444548,0.289743


In [ ]:
dump(scaled_demand, open('scaler_demand.pkl', 'wb'))

In [ ]:
Y.head()

,Date,system_cost_70,CO2_costs_70,startup_cost_70,shutdown_cost_70,Coal_generation_70,NG_generation_70,Nuclear_generation_70,Biomass_Generation_70,Hydro_Generation_70,...,Unnamed: 20,system_cost_76,CO2_costs_76,startup_cost_76,shutdown_cost_76,Coal_generation_76,NG_generation_76,Nuclear_generation_76,Biomass_Generation_76,Hydro_Generation_76
0,01/01/2019,5048325.624,3170756.506,2496.66,280.89,94433.69140,0.000000,38880.0,117.5,418791.0900,...,NaN,3317791.185,2.113245e+06,1841.63,228.44,59371.48761,3.824738e+02,38880.00000,102.500000,411910.7457
1,01/02/2019,3984855.561,2561109.776,2393.76,288.88,74257.71598,700.000000,38880.0,62.5,402022.5804,...,NaN,1456985.612,9.464519e+05,1358.89,188.03,21919.75447,2.935528e+02,38880.00000,62.334683,364555.4341
2,01/03/2019,4930318.514,3187960.095,2521.23,305.32,94112.94477,1572.695301,38880.0,62.5,419125.9945,...,NaN,1437274.098,9.039136e+05,1355.87,157.42,20226.02478,5.732840e+02,38476.41334,58.755960,370203.8759
3,01/04/2019,4364269.371,2886076.697,2398.82,143.72,84826.53506,620.417839,38880.0,72.5,411868.3590,...,NaN,1416639.318,9.318883e+05,1035.88,102.21,21536.94775,6.170000e-12,38624.80665,0.000000,367652.2216
4,01/05/2019,3882769.811,2519133.191,1961.09,226.10,72781.02942,612.000000,38880.0,112.5,407635.3987,...,NaN,1249865.528,8.144984e+05,905.48,116.38,17823.33009,0.000000e+00,38542.07932,0.000000,360429.3706


In [ ]:
Y_70 = Y.CO2_costs_70.copy()
Y_73 = Y.CO2_costs_73.copy()
Y_76 = Y.CO2_costs_76.copy()

Y_70.head()

0    3170756.506
1    2561109.776
2    3187960.095
3    2886076.697
4    2519133.191
Name: CO2_costs_70, dtype: float64

### Normalizing Weather Variables of 70C



In [ ]:
'''70C
df_avg_70_DB, dry bulb temperature
df_avg_70_OD, outdoor dewpoint
df_avg_70_HR, humidity ratio
df_avg_70_PV, PV production
'''

from sklearn import preprocessing
hours = 24

db_column_names = ["db_hour_{}".format(x) for x in range(hours)]
od_column_names = ["od_hour_{}".format(x) for x in range(hours)]
hr_column_names = ["humid_hour_{}".format(x) for x in range(hours)]
pv_column_names = ["pv_hour_{}".format(x) for x in range(hours)]

scaler= preprocessing.MinMaxScaler()
d_db_70 = scaler.fit_transform(df_avg_70_DB.T)
dump(scaler, open('scaled_db_70.pkl', 'wb'))
scaled_db_70= pd.DataFrame(d_db_70, columns= db_column_names)

d_od_70 = scaler.fit_transform(df_avg_70_OD.T)
dump(scaler, open('scaled_od_70.pkl', 'wb'))
scaled_od_70= pd.DataFrame(d_od_70, columns= od_column_names)

d_hr_70= scaler.fit_transform(df_avg_70_HR.T)
dump(scaler, open('scaled_hr_70.pkl', 'wb'))
scaled_hr_70 = pd.DataFrame(d_hr_70, columns=hr_column_names)

d_pv_70= scaler.fit_transform(df_avg_70_PV.T)
dump(scaler, open('scaled_pv_70.pkl', 'wb'))
scaled_pv_70 = pd.DataFrame(d_pv_70, columns=pv_column_names)

In [ ]:
degree_70 = 70
degree_73 = 73
degree_76 = 76

max_degree = 76
min_degree = 70

In [ ]:
x_70_scaled = pd.concat([scaled_demand, scaled_db_70, scaled_od_70, scaled_hr_70, scaled_pv_70], axis=1)
x_70_scaled['degree'] = (degree_70 - min_degree) / (max_degree - min_degree)
x_70_scaled.head()

,demand_hr_0,demand_hr_1,demand_hr_2,demand_hr_3,demand_hr_4,demand_hr_5,demand_hr_6,demand_hr_7,demand_hr_8,demand_hr_9,...,pv_hour_15,pv_hour_16,pv_hour_17,pv_hour_18,pv_hour_19,pv_hour_20,pv_hour_21,pv_hour_22,pv_hour_23,degree
0,0.250408,0.383149,0.288236,0.365444,0.369953,0.409463,0.566915,0.599055,0.330089,0.361597,...,0.350678,0.301901,0.243496,0.017058,0.0,0.000466,0.0,0.0,0.0,0.0
1,0.220139,0.268730,0.278986,0.287109,0.350493,0.274975,0.460179,0.391473,0.312463,0.269490,...,0.240245,0.293585,0.081054,0.000000,0.0,0.000466,0.0,0.0,0.0,0.0
2,0.437109,0.357765,0.529862,0.335294,0.656674,0.304670,0.713162,0.409791,0.429430,0.429872,...,0.421859,0.263625,0.134165,0.001010,0.0,0.000466,0.0,0.0,0.0,0.0
3,0.420069,0.301355,0.530066,0.289022,0.608912,0.284047,0.630784,0.379567,0.344742,0.254582,...,0.200751,0.294907,0.219923,0.007252,0.0,0.000466,0.0,0.0,0.0,0.0
4,0.403754,0.252320,0.436449,0.265804,0.515776,0.262421,0.558195,0.334192,0.373130,0.252489,...,0.114504,0.149666,0.168306,0.049991,0.0,0.000466,0.0,0.0,0.0,0.0


In [ ]:
#Y_70.values
y_70 = Y_70.to_numpy()
x_70 = x_70_scaled.to_numpy()

### Normalizing Weather Variables of 73C





In [ ]:
from pickle import load

# split data into train and test sets
#_, X_test, _, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# load the model
scaler_db_70 = load(open('scaled_db_70.pkl', 'rb'))
scaler_od_70 = load(open('scaled_od_70.pkl', 'rb'))
scaler_hr_70 = load(open('scaled_hr_70.pkl', 'rb'))
scaler_pv_70 = load(open('scaled_pv_70.pkl', 'rb'))

d_db_73 = scaler_db_70.transform(df_avg_73_DB.T)
scaled_db_73= pd.DataFrame(d_db_73, columns= db_column_names)

d_od_73 = scaler_od_70.fit_transform(df_avg_73_OD.T)
scaled_od_73= pd.DataFrame(d_od_73, columns= od_column_names)

d_hr_73= scaler_hr_70.fit_transform(df_avg_73_HR.T)
scaled_hr_73 = pd.DataFrame(d_hr_70, columns=hr_column_names)

d_pv_73= scaler_pv_70.fit_transform(df_avg_73_PV.T)
scaled_pv_73 = pd.DataFrame(d_pv_70, columns=pv_column_names)

In [ ]:
#checking scaler values
scaler_db_70.scale_

array([0.02495757, 0.02426007, 0.02469136, 0.02493517, 0.02502503,
       0.02384359, 0.02382314, 0.02374169, 0.0244093 , 0.02498002,
       0.02534469, 0.0262302 , 0.02635463, 0.02555453, 0.02511553,
       0.02473533, 0.02449539, 0.02356046, 0.02314815, 0.02320616,
       0.02358046, 0.02358046, 0.02426007, 0.0250025 ])

In [ ]:
x_73_scaled = pd.concat([scaled_demand, scaled_db_73, scaled_od_73, scaled_hr_73, scaled_pv_73], axis=1)
x_73_scaled['degree'] = (degree_73 - min_degree) / (max_degree - min_degree)
x_73_scaled.head()

,demand_hr_0,demand_hr_1,demand_hr_2,demand_hr_3,demand_hr_4,demand_hr_5,demand_hr_6,demand_hr_7,demand_hr_8,demand_hr_9,...,pv_hour_15,pv_hour_16,pv_hour_17,pv_hour_18,pv_hour_19,pv_hour_20,pv_hour_21,pv_hour_22,pv_hour_23,degree
0,0.250408,0.383149,0.288236,0.365444,0.369953,0.409463,0.566915,0.599055,0.330089,0.361597,...,0.350678,0.301901,0.243496,0.017058,0.0,0.000466,0.0,0.0,0.0,0.5
1,0.220139,0.268730,0.278986,0.287109,0.350493,0.274975,0.460179,0.391473,0.312463,0.269490,...,0.240245,0.293585,0.081054,0.000000,0.0,0.000466,0.0,0.0,0.0,0.5
2,0.437109,0.357765,0.529862,0.335294,0.656674,0.304670,0.713162,0.409791,0.429430,0.429872,...,0.421859,0.263625,0.134165,0.001010,0.0,0.000466,0.0,0.0,0.0,0.5
3,0.420069,0.301355,0.530066,0.289022,0.608912,0.284047,0.630784,0.379567,0.344742,0.254582,...,0.200751,0.294907,0.219923,0.007252,0.0,0.000466,0.0,0.0,0.0,0.5
4,0.403754,0.252320,0.436449,0.265804,0.515776,0.262421,0.558195,0.334192,0.373130,0.252489,...,0.114504,0.149666,0.168306,0.049991,0.0,0.000466,0.0,0.0,0.0,0.5


In [ ]:
y_73 = Y_73.to_numpy()
x_73 = x_73_scaled.to_numpy()

### Normalizing Weather Variables of 76C

In [ ]:
d_db_76 = scaler_db_70.transform(df_avg_76_DB.T)
scaled_db_76= pd.DataFrame(d_db_76, columns= db_column_names)

d_od_76 = scaler_od_70.fit_transform(df_avg_76_OD.T)
scaled_od_76 = pd.DataFrame(d_od_76, columns= od_column_names)

d_hr_76= scaler_hr_70.fit_transform(df_avg_76_HR.T)
scaled_hr_76 = pd.DataFrame(d_hr_76, columns=hr_column_names)

d_pv_76 = scaler_pv_70.fit_transform(df_avg_76_PV.T)
scaled_pv_76 = pd.DataFrame(d_pv_76, columns=pv_column_names)
#Repeat the steps

In [ ]:
x_76_scaled = pd.concat([scaled_demand, scaled_db_76, scaled_od_76, scaled_hr_76, scaled_pv_76], axis=1)
x_76_scaled['degree'] = (degree_76 - min_degree) / (max_degree - min_degree)
x_76_scaled.head()

,demand_hr_0,demand_hr_1,demand_hr_2,demand_hr_3,demand_hr_4,demand_hr_5,demand_hr_6,demand_hr_7,demand_hr_8,demand_hr_9,...,pv_hour_15,pv_hour_16,pv_hour_17,pv_hour_18,pv_hour_19,pv_hour_20,pv_hour_21,pv_hour_22,pv_hour_23,degree
0,0.250408,0.383149,0.288236,0.365444,0.369953,0.409463,0.566915,0.599055,0.330089,0.361597,...,0.350678,0.301901,0.243496,0.017058,0.0,0.000466,0.0,0.0,0.0,1.0
1,0.220139,0.268730,0.278986,0.287109,0.350493,0.274975,0.460179,0.391473,0.312463,0.269490,...,0.240245,0.293585,0.081054,0.000000,0.0,0.000466,0.0,0.0,0.0,1.0
2,0.437109,0.357765,0.529862,0.335294,0.656674,0.304670,0.713162,0.409791,0.429430,0.429872,...,0.421859,0.263625,0.134165,0.001010,0.0,0.000466,0.0,0.0,0.0,1.0
3,0.420069,0.301355,0.530066,0.289022,0.608912,0.284047,0.630784,0.379567,0.344742,0.254582,...,0.200751,0.294907,0.219923,0.007252,0.0,0.000466,0.0,0.0,0.0,1.0
4,0.403754,0.252320,0.436449,0.265804,0.515776,0.262421,0.558195,0.334192,0.373130,0.252489,...,0.114504,0.149666,0.168306,0.049991,0.0,0.000466,0.0,0.0,0.0,1.0


In [ ]:
y_76 = Y_76.to_numpy()
x_76 = x_76_scaled.to_numpy()

RandomForest Model

In [ ]:
#scaler = preprocessing.MinMaxScaler()
#matrix_scaled = scaler.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(x_70, y_70, test_size=.2, random_state = 42)

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, Y_train)

results = model.predict(X_test)

r2 = r2_score(Y_test, results)
print(r2)


0.9889970356529931
38271456648.73568


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x_73, y_73, test_size=.2, random_state = 42)

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, Y_train)

results = model.predict(X_test)

r2 = r2_score(Y_test, results)
print(r2)

0.9776241315992283
46649530796.60375


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x_76, y_76, test_size=.2, random_state = 42)

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, Y_train)

results = model.predict(X_test)

r2 = r2_score(Y_test, results)
print(r2)

0.8767464035028295
122324740819.71437


In [ ]:
results

In [ ]:
Y_test